In [1]:
#from audioModel import predict, train, accuracy, getCorrectAndIncorrect
#from audioInput import getLaughTracks
import tensorflow as tf
from audioModel import predict, accuracy, getCorrectAndIncorrect
from audioInput import readFolderRecursive
from audioDisplay import getModel, displayAudio
from audioInput_v2 import gatherTestingData, getOneHot, concatSamples
from audioDisplay import printResults
import numpy as np

In [11]:
dirs = ['samples/test-samples/guitar','samples/test-samples/piano']
classes = ['guitar', 'piano']
model_name = 'guitar-piano'


dirs = ['samples/test-samples/orchestra-test','samples/test-samples/wolves-test']
classes = ['orchestra', 'wolves']
model_name = 'orchestra-wolves'

In [12]:
files = []
for d in dirs:
    files += readFolderRecursive(d)  
    
x, chunks = gatherTestingData(files)

model_name = getModel(model_name)
preds = predict(model_name, len(classes), x)


100%|██████████| 2/2 [00:00<00:00, 26.83it/s]89it/s]

im punting on this and just discarding chunks that dont have audio



/notebooks/laugh-tracks/audioInput.py:123: RuntimeWarning: invalid value encountered in true_divide
  normalized = (x-np.min(x))/(np.max(x)-np.min(x))


INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/orchestra-wolves/20_20/model


In [13]:

def getLabelsFromChunks(chunks):
    labels = {}
    for chunk in chunks:
        label = chunk['file'].split('/')[2:3][0]
        i = len(labels.keys()) - 1
        #print('label', label)
        labels[label] = i
        
    one_hot_labels = None
    classes = len(labels.keys())
    #print('classes', classes)
    for chunk in chunks:
        label = chunk['file'].split('/')[2:3][0]
        #print('label', label)
        #print('labels', labels[label])
        one_hots = getOneHot(classes, labels[label])
        if one_hot_labels is None:
            one_hot_labels = [one_hots]
        else:
            #print('one hots', one_hots)
            one_hot_labels = np.concatenate((one_hot_labels, [one_hots]), axis=0)
    #print('final one hots', one_hot_labels)
    return one_hot_labels

def displayCorrectIncorrectResults(preds, labels, chunks): 
    #print(preds, labels, chunks)
    pretty = printResults(preds)
    prettyLabels = printResults(labels)
    #for i in range(len(preds)):
        #print('actual', prettyLabels[i], 'predicted (pretty)', pretty[i], 'predicted', preds[i])

    correct = []
    incorrect = []
    #print(preds, labels)
    print('accuracy', accuracy(preds, labels))
    correct, incorrect = getCorrectAndIncorrect(preds, labels)
    #print(correct[0:5])
    for result in incorrect[0:5]:
        i = result['i']
        print(i, chunks[i])

    #return correct, incorrect
displayCorrectIncorrectResults(preds, getLabelsFromChunks(chunks), chunks)

#printResults2(preds, ])




accuracy 0.8992805755395683
123 {'starting_index': 118819, 'audio': <pydub.audio_segment.AudioSegment object at 0x7f4641057748>, 'file': 'samples/test-samples/wolves-test/QnVzn-1mESU.wav', 'samples': array([[[0.2392311 , 0.34770274, 0.51486777, ..., 0.51243761,
         0.59934854, 0.70222001],
        [0.46372083, 0.44465184, 0.44107387, ..., 0.47485834,
         0.4958912 , 0.72598007],
        [0.43714216, 0.51186162, 0.54418337, ..., 0.49751656,
         0.50176938, 0.62949127],
        ...,
        [0.35299844, 0.35800754, 0.2672055 , ..., 0.32611577,
         0.33286176, 0.3274004 ],
        [0.21894255, 0.2048163 , 0.30118299, ..., 0.3254897 ,
         0.25945579, 0.22095982],
        [0.21747328, 0.2710391 , 0.31931507, ..., 0.41188453,
         0.31797685, 0.30651352]]])}
267 {'starting_index': 258211, 'audio': <pydub.audio_segment.AudioSegment object at 0x7f4641000dd8>, 'file': 'samples/test-samples/wolves-test/QnVzn-1mESU.wav', 'samples': array([[[0.38642023, 0.39303583, 0.3

In [7]:
chunks[123]['audio']